# **Reservoir Computing**

Introduced by Jaeger and Haas (2004), a reservoir computer is a type of Recurrent Neural Network (RNN). However, unlike traditional RNNs, an RC is initialized randomly and remains unaltered throughout training. This means it does not necessarily undergo backpropagation or gradient descent.

RC primarily focuses on its output layer. The outputs from the reservoir nodes are linearly aggregated to yield the expected outcome, making the training process similar to that of a linear regression model.

The readout layer (or output) weights are fine-tuned to minimize the discrepancy between the RC's predictions and the actual outputs.

**Mathematically:**

Let $u(t)$ be an input vector that is fed into a reservoir $R$ through an input (Reservoir coupler). The input to $R$ then becomes,

$W_{in} u(t)$

Where

*  $W_{in} \in ℝ ^{D_r×D} $
*  $D_r$ is the dimension of the reservoir, and
*  $D$ is the dimension of the input.

In $R$ the input is then combined with the current reservoir state, $r(t)$ to produce the output (or the next reservoir state) $r(t+dt)$.

The reservoir is represented by an adjacency matrix *A* where:

*  $A ∈ ℝ^{D_r×D}$ and
*  $D_r$ is the dimension of the reservoir.

Therefore:

$r(t+dt) = tanh[Ar(t)+W_{in}u(t)]$

Each reservoir node has a corresponding reservoir state, and $r(t+dt)$ flows into the reservoir.

Here, a mapping is done, given by:

$v(t+dt) = r(t+dt), W_{out})$

Where:

*  $v(t+dt) ∈ ℝ^D $, and
*  $W_{out} ∈ ℝ^{D×D_r}$

**The goal is for the output $v(t)$ to approximate the actual desired output $v'(t)$.**

During the training process $(-T ≤ t < 0)$. An input $u(t)$ is fed into the reservoir, and the resulting reservoir state $r(t)$ is then stored along with $u(t)$ as training data.

**The parameters of $W_{out}$ are then trained to minimize the mean squared difference between $v(t)$ and $v'(t)$** given by:

$∑_{t=t_0}^{T}||(r(t),W_{out})-v'_d(t)||^2+ β ||W_{out}||^2$

Where beta corresponds to the regularization constant and prevents overfitting if $β>0$.

To transition from a training phase to the prediction phase, we let $v(t+dt)=u(t+dt)$. This corresponds to letting the output vector $v(t+dt)$ serve as the input for the next timestamp $u(t+dt)$.

The neural network is then allowed to run autonomously according to:

$r(t+dt) =tanh [Ar(t) +W_{in} (r(t), W_{out})]$

Therefore, the output of the "trained" RC gives the input (predicted value) for the next timestep $u(t)$ for $t>0$.

## **Defining the Activation Functions**

### **Traditionanal Functions**

In [18]:
def sigmoid(x):
  """
  Sigmoid activation function that transforms input values to range (0,1).

  Parameters:
    x: Input array or value

  Returns:
    Sigmoid transformation of input
  """
  return 1 / (1 + np.exp(-x))

def tanh(x):
    """
    Hyperbolic tangent activation function that transforms input values to range (-1,1).

    Parameters:
        x: Input array or value

    Returns:
        Hyperbolic tangent transformation of input
    """
    return np.tanh(x)

def relu(x):
    """
    Rectified Linear Unit (ReLU) activation function.
    Returns x if x > 0, otherwise returns 0.

    Parameters:
        x: Input array or value

    Returns:
        ReLU transformation of input
    """
    return np.maximum(0, x)

### **Adapted ReLU Functions**

In [19]:
def leaky_relu(x, alpha=0.01):
    """
    Leaky ReLU activation function.
    Returns x if x > 0, otherwise returns alpha * x.

    Parameters:
        x: Input array or value
        alpha: Slope for negative values (default: 0.01)

    Returns:
        Leaky ReLU transformation of input
    """
    return np.where(x > 0, x, alpha * x)

def elu(x, alpha=1.0):
    """
    Exponential Linear Unit (ELU) activation function.
    Returns x if x > 0, otherwise returns alpha * (exp(x) - 1).

    Parameters:
        x: Input array or value
        alpha: Scaling factor for negative values (default: 1.0)

    Returns:
        ELU transformation of input
    """
    return np.where(x > 0, x, alpha * (np.exp(x) - 1))

def prelu(x, alpha):
    """
    Parametric ReLU (PReLU) activation function.
    Returns x if x > 0, otherwise returns alpha * x.
    The alpha parameter is learned during training.

    Parameters:
        x: Input array or value
        alpha: Learned parameter array with same shape as x

    Returns:
        PReLU transformation of input
    """
    return np.where(x > 0, x, alpha * x)

## **Define Helper Functions**

In [11]:
from scipy import linalg

def linear_regression(X, Y):
    """
    Computes linear regression weights for mapping reservoir states to target outputs.
    Solves the equation W_out * X = Y for W_out using least squares.

    Parameters:
        X: Matrix of reservoir states (dim_reservoir × n_samples)
        Y: Matrix of target outputs (n_samples × dim_system)

    Returns:
        W_out: Output weight matrix (dim_system × dim_reservoir)
    """
    # Transpose X to match the dimensions expected in the solve
    X_T = X.T

    # Solve the linear system for W_out.T
    # We're solving X_T @ W_out.T = Y
    # Using numpy's least squares solver for better numerical stability
    W_out_T = linalg.lstsq(X_T, Y)[0]

    # Return the transpose to get W_out
    return W_out_T.T

def generate_adjacency_matrix(dim_reservoir, rho, sigma, density=0.1):
    """
    Generates a random adjacency matrix for the reservoir.

    Parameters:
        dim_reservoir: Size of the reservoir
        rho: Spectral radius - controls the dynamics of the reservoir
        sigma: Scaling factor for weights
        density: Fraction of non-zero connections (default 0.1)

    Returns:
        A: Adjacency matrix for the reservoir
    """
    # Generate a sparse random matrix
    A = np.random.rand(dim_reservoir, dim_reservoir)

    # Apply sparsity by setting values below threshold to zero
    A[A < (1 - density)] = 0

    # Scale the matrix
    A = A * 2 * sigma - sigma

    # Use the scale_matrix function instead of direct scaling
    A = scale_matrix(A, rho)

    return A

def scale_matrix(A, rho):
    eigenvalues, _ = np.linalg.eig(A)
    max_eigenvalue = np.amax(eigenvalues)
    A = A/np.absolute(max_eigenvalue) * rho
    return A

## **Define Reservoir Computer Class**

In [12]:
!pip install networkx

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [14]:
class ReservoirComputer:
  def __init__ (self, dim_system, dim_reservoir, rho, sigma, density,
                activation_function):
        # dim_system = dimension of the input / output system
        # dim_reservoir = size of the reservoir (number of neurons)
        # rho = spectral radius parameter that affects the dynamics
        # sigma = scale parameter for weight initialization
        # density = controls the sparsity of the reservoir connections
        # activation_function = the activation function; sigmoid = default
    self.dim_system = dim_system
        # stores the system dimensions as variables
    self.dim_reservoir = dim_reservoir
        # stores the reservoir dimensions as variables
    self.r_state = np.zeros(dim_reservoir)
        # initializes the reservoir state as a vector of zeros with length equal
        # to the reservoir dimension
    self.A = generate_adjacency_matrix(dim_reservoir, rho, sigma, density)
        # creates the reservoir's internal connection matrix using a function
    self.W_in = 2 * sigma * (np.random.rand(dim_reservoir, dim_system) - 0.5)
        # creates the input weight matrix (connecting inputs to the reservoir)
        # with random values between -σ and σ
    self.W_out = 2 * sigma * (np.random.rand(dim_system, dim_reservoir))
        # initializes an output weight matrix (connecting the reservoir to the output)
        # with random values between 0 and 2σ
    self.activation_function = activation_function #store the activation function

  def advance_r_state(self, u):
        # updates the reservoir state based on the current input u
    self.r_state = self.activation_function(np.dot(self.A, self.r_state) + np.dot(
        self.W_in, u))
        # updates the reservoir state using:
          # the current state multiplied by the adjacency matrix
          # plus the input weighted by the input matrix
          # then applies the sigmoid activation function
            # use any activation function here - sigmoid is a placeholder
    return self.r_state
        # returns the new reservoir state

  def v(self):
        # computes the output of the reservoir computer
    return np.dot(self.W_out, self.r_state)
        # the output is calculated by multiplying the output weight matrix with
        # the current reservoir state

  def train(self, trajectory):
    # trajectory has shape (n, dim_system), n is the number of timesteps
        # so... the parameter is a 2D array with shape (n, dim_system), where n is
        # the number of time steps
    R = np.zeros((self.dim_reservoir, trajectory.shape[0]))
        # creates matrix R to store the reservoir states for each trajectory timestep
    for i in range(trajectory.shape[0]):
        # loops through each timestep in the trajectory
      R[:, i] = self.r_state
        # stores the current reservoir state in the i-th comumn of R
      u = trajectory[i]
        # gets the current input from the trajectory
      self.advance_r_state(u)
        # updates the reservoir state using the current input
    self.W_out = linear_regression(R, trajectory)
        # computes the output weights using linear regression between the collected
        # reservoir states (R) and the target trajectory

  def predict(self, steps):
    prediction = np.zeros((steps, self.dim_system))
        # creates an array to store the predictioons with shape (steps, dim_system)
    for i in range(steps):
        # loops through each prediction step
        v = self.v() # computes the current output in the prediction array
        prediction[i] = v # updates the reservoir state using the current output
          # (feeding back the prediction)
        self.advance_r_state(v)
    return prediction

### **Plot the Results**

**(Lorenz-63 Attractor)**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # Import for 3D plotting

# Assuming valid_data and pred_data are already defined
# Example (remove if you already have the data):
# valid_data = np.array(...)  # Ground truth data
# pred_data = np.array(...)   # Prediction data

# Create a 3D figure
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Extract coordinates from the arrays
x_data = valid_data[:, 0]
y_data = valid_data[:, 1]
z_data = valid_data[:, 2]

x_pred = pred_data[:, 0]
y_pred = pred_data[:, 1]
z_pred = pred_data[:, 2]

# Plot the actual trajectory
ax.plot(x_data, y_data, z_data, lw=0.5, label="Actual", color='blue')

# Plot the predicted trajectory
ax.plot(x_pred, y_pred, z_pred, lw=0.5, label="Predicted", color='red')

# Add labels and title
ax.set_xlabel("X Axis")
ax.set_ylabel("Y Axis")
ax.set_zlabel("Z Axis")

# Add legend and title
ax.legend()
ax.set_title("Lorenz Attractor")

# Adjust the view angle for better visualization
ax.view_init(elev=30, azim=45)

# Show the plot
plt.tight_layout()
plt.show()

**Predicted Data vs Actual Data as a Function of Time**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming these variables are already defined:
# training_data, data_length, x_data, y_data, z_data, x_pred, y_pred, z_pred

# Create a figure with 3 subplots stacked vertically
fig, axs = plt.subplots(3, 1, figsize=(8, 10))

# Define the timesteps for the x-axis
timesteps = range(training_data.shape[0], data_length)

# Plot X dimension comparison
axs[0].plot(timesteps, x_data, label="x", lw=1.1, color='blue')
axs[0].plot(timesteps, x_pred, label="x_pred", color='orange', lw=1.1)
axs[0].set_ylabel("x")
axs[0].set_title("Comparison of the Predicted Lorenz Data and Actual Data")
axs[0].legend()

# Plot Y dimension comparison
axs[1].plot(timesteps, y_data, label="y", color='r', lw=1.1)
axs[1].plot(timesteps, y_pred, label="y_pred", color='orange', lw=1.1)
axs[1].set_ylabel("y")
axs[1].legend()

# Plot Z dimension comparison
axs[2].plot(timesteps, z_data, label="z", color='g', lw=0.9)
axs[2].plot(timesteps, z_pred, label="z_pred", color='orange', lw=1.1)
axs[2].set_ylabel("z")
axs[2].set_xlabel("timestep")
axs[2].legend()

# Adjust spacing between subplots
fig.tight_layout()

# Display the plot
plt.show()